In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
import time
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from tqdm.notebook import tqdm

In [2]:
key_list = pd.read_csv("../../법정동코드 전체자료.txt", sep='\t', encoding='CP949')
key_list = key_list.법정동명[key_list.폐지여부=='존재'].str.split(n=2).str.get(1).unique()[1:]

In [11]:
len(key_list[pd.Series(key_list).str.endswith(('구', '동', '시', '군'))])

221

In [12]:
len(key_list)

231

## 지역을 2번째 (특별시는 OO구, 도는 OO시 또는 OO군)로 했을때 231개

In [170]:
key_list

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구',
       '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구',
       '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '미추홀구', '연수구', '남동구',
       '부평구', '계양구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군', '반곡동',
       '소담동', '보람동', '대평동', '가람동', '한솔동', '나성동', '새롬동', '다정동', '어진동',
       '종촌동', '고운동', '아름동', '도담동', '세종특별자치시', '조치원읍', '연기면', '연동면', '부강면',
       '금남면', '장군면', '연서면', '전의면', '전동면', '소정면', '수원시', '성남시', '의정부시',
       '안양시', '부천시', '광명시', '평택시', '동두천시', '안산시', '고양시', '과천시', '구리시',
       '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '파주시', '이천시',
       '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군',
       '양평군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군',
       '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군',
       '양양군

In [34]:
options = webdriver.chrome.options.Options()
options.add_argument('headless')
options.add_argument('incognito')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('../../chromedriver', options=options)
driver.set_window_size(1000,800)
driver.implicitly_wait(0.5)
driver.get("https://www.mangoplate.com/")
time.sleep(3)

In [11]:
review_df = pd.DataFrame(columns=['Region', 'Rating', 'Category', 'Review'])

In [6]:
rating_dict = {'맛있다':3, '괜찮다':2, '별로':1}

In [13]:
for keyword in key_list:
    search_time = time.time()
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p > a"):
        page.click()
        time.sleep(1)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])
            
            more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
            
            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
            region = info_li[0].find_element_by_css_selector("td > span.Restaurant__InfoAddress--Text").text
            region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
            category = info_li[2].find_element_by_css_selector("td").text

            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

종로구
종로구 : 7488.437269687653 초 소요
중구


ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=93.0.4577.82)


In [33]:
review_df

,Region,Rating,Category,Review
0,종로구,3,한정식 / 백반 / 정통 한식,"- 한줄평: 왜 이렇게 평점이 높은지 이해가 가는 집\n- 주문 메뉴: 병어찜 중,..."
1,종로구,3,한정식 / 백반 / 정통 한식,“얼마나 맛있는지 알지 못 할거야”
2,종로구,1,한정식 / 백반 / 정통 한식,"먹으면서도 혼란스러웠고, 리뷰를 쓰는 지금도 혼란스럽다. 어떤 음식점이든 호불호가 ..."
3,종로구,1,한정식 / 백반 / 정통 한식,비린거 싫어하는 사람은 먹지마세요
4,종로구,3,한정식 / 백반 / 정통 한식,#유명세가아깝지않은맛집\n\n유명하지만 다행히도 평일 점심 기준 자리 여유있는편\n...
...,...,...,...,...
12545,종로구,3,카페 / 디저트,커피 맛있게 먹은 기억은 있는데.. 공간이 너무 불편하고 사람도 바글바글해서 아쉬워...
12546,종로구,3,카페 / 디저트,"커피 하루에 한잔만 마시는데, 다른곳에서 커피 마시다가 펠트카페를 보고 고민고민 하..."
12547,종로구,3,카페 / 디저트,광화문 디타워 지하에도 펠트가 생겼어요. 본점보다 좀더 세련되고 정제된 느낌이에요....
12548,종로구,3,카페 / 디저트,"여러분 빨리 가세요,,,제 퇴근길의 생명줄이되었읍니다....보장된 펠트적 커피맛과 ..."


종로구 완료. 중구에서 막혀서 중구부터 다시

구 바뀔때, get /search/keyword 하고, sleep 필요

In [48]:
driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(1)").text

'주소 서울시 강북구 수유동 산 17-1'

In [61]:
driver.get("https://www.mangoplate.com/search/강북구")

In [72]:
each_restaurant = driver.find_elements_by_css_selector("a > span.title.ng-binding")[0]

In [73]:
ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
driver.switch_to.window(driver.window_handles[-1])

In [95]:
driver.find_element_by_css_selector("section.restaurant-detail > table > tbody > tr:nth-child(3) > td").text

'기타 중식'

In [94]:
'a c a\nb'.split('\n')[0].split()[1]

'c'

In [46]:
for keyword in key_list[1:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
            region = info_li[0].find_element_by_css_selector("td").text.split('\n')[0]
            region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
            category = info_li[2].find_element_by_css_selector("td").text

            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

중구
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="8e9ffcba-b8a0-4c00-aee4-5ee6dc35c21a")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="3e2654dc-43db-4fb2-bd7f-dedf85ec4893")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="94941f88-0370-4fc3-9434-6a1ae7e8fca1")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="a91c2e58-79ad-404f-8a84-ca5ba85ee585")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="e7b12403-0ef4-450f-a946-c590794e7126")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="b6d97c80-db13-43c3-82a3-4ce540d2e668")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="84586f83-e1d0-4e37-ae81-08b6474188ec

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="1c132054-949b-4de1-857e-447a29e8a64b")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="121e7119-8385-4ded-913b-91b7201a1f3d")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="8b02eecf-1847-4597-b8ea-1cd29da9ccc6")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="9eececcb-5e6c-4574-b52a-a9610ea503ab")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="ee9926d3-295c-4b3c-bcc6-67c3fca68f4a")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="40a674d5-c210-4a45-b9ba-2389f06e206c")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="41249e9f-de5e-48f0-958e-3b4711317a37")>

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="45d9a359-1a11-425a-bfe8-0cf77a4f1149")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="435d2b14-0202-4f08-a821-6e1315008171")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="a07fed07-b656-46b9-8faa-f7e10a31a95e")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="7c757d0b-1f8c-4a06-8a6a-af71b40f8710")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="986839d6-8822-4c16-b3b6-422aaa32b17e")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="103ce5e9-4b64-49ac-a71e-6e23e6fc5719")>


Exception ignored in: <function tqdm.__del__ at 0x0000019B91629CA0>
Traceback (most recent call last):
  File "C:\Users\dongi\anaconda3\envs\dacon\lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
  File "C:\Users\dongi\anaconda3\envs\dacon\lib\site-packages\tqdm\notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="5a9428f3-528b-4808-b79b-5891c7cc881c")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="08c42665-e482-4e1e-ade9-0c6389b65afa")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="94ae2a81-e27c-4aa8-9d1c-1eea47934dad")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="86d68fbe-8f18-4a7b-b224-42729d5179ac")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="135eeec2-3e5c-4ebc-99b1-80c6e4094ed6")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="9e9b78af-dccc-4a7c-be3d-204b4b08b53b")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="478d7756-0005-45a0-8da5-966c2175f654")>

...

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="bc0b22c7-7c84-4074-8b90-cd367cba08f1")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="8330a4f0-de73-4fcb-bd4b-191ef82f66b1")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="9da4113a-4fbb-4642-b64b-00e334315e74")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="afa54858-d62f-4486-87a6-2f9f28005e6e")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="dd61f2db-8cb3-44a6-8762-74f94c289091")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="26382233-9830-444f-803d-4aace555ef2c")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="644b527e-c5a6-4a58-85e7-fd8d95db2bf3")>

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="44ac105d-297c-4b19-8efd-b1bd435330cd")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="93e58874-f74a-4cd4-a737-bb412c3f1f15")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="f47d612d-f3ce-45d0-8a81-bbf722a1894c")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="8b869196-59d4-4fbc-a984-773ae2a915c7")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="f7f84605-38bf-415a-abf1-8252787ac6e3")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="61fbe86c-1983-4001-9c41-6c589054ec20")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="b0e87503-063b-4557-83b0-e136c1d89794")>

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="f68d7976-7149-4bd9-9213-bded60f16f8d")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="8ffdcc6a-e269-4d72-b726-133ea4273432")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="294289ec-803c-4b56-9b7f-b3a80c61ef70")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="3bf126fa-b912-4b00-8167-e7892baad1c4")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="a972dfdc-2cfd-4efc-a7c6-803c2fc7695c")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="e4d80555-3787-4a6c-995f-5e2e062917a7")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="f3fa07ee-6cfe-4803-a08c-aff2cbb28c13")>

<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="1203c22b-9c55-4e4e-a19a-791229b9f66e")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="50485bff-4a27-4ab7-ab8f-de305b3c79a8")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="23b18d03-9a26-4e95-bbe2-48ecea07e319")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="15cf917f-62e3-43bb-8abe-3ce31ca8fdbc")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="fe8c4505-89c5-464a-ad90-f21c3635d864")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="2a99eba1-85ec-4b80-969e-58ece161a61d")>
<selenium.webdriver.remote.webelement.WebElement (session="cf4061e4e65995d87b6a582979928c1c", element="52593b95-d204-49b4-ad70-7684d3d9c73d")>

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td > span.Restaurant__InfoAddress--Text"}
  (Session info: headless chrome=93.0.4577.82)


강북구 부터 다시. 주소를 지번 말고 도로명으로. 지번이 없는 곳도 있음. 이름:북한산 인수재

In [52]:
review_df = review_df.drop(index=review_df[review_df.Region=='강북구'].index)

In [103]:
review_df.to_parquet("./key0-7.parquet")

In [105]:
for keyword in key_list[8:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
            region = info_li[0].find_element_by_css_selector("td").text.split('\n')[0]
            region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
            category = info_li[2].find_element_by_css_selector("td").text

            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

강북구
강북구 : 10044.50998210907 초 소요
도봉구
도봉구 : 6655.941582202911 초 소요
노원구
노원구 : 7386.822394371033 초 소요
은평구
은평구 : 2900.086626768112 초 소요
서대문구


IndexError: list index out of range

서대문구 스탑. region의 out of range list? 모르겠다. 소신이쏘 음식점.

In [112]:
review_df.Rating.value_counts()

3    46001
2    14003
1     4534
Name: Rating, dtype: int64

In [121]:
review_df = review_df.drop(index=review_df[review_df.Region=='서대문구'].index)

In [117]:
review_df.to_parquet("./key0-11.parquet")

In [128]:
for keyword in key_list[12:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
            region = info_li[0].find_element_by_css_selector("td").text.split('\n')[0]
            region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
            category = info_li[2].find_element_by_css_selector("td").text

            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

서대문구
서대문구 : 7927.927355051041 초 소요
마포구
마포구 : 8487.02433013916 초 소요
양천구
양천구 : 6315.5415115356445 초 소요
강서구
강서구 : 5197.271981477737 초 소요
구로구
구로구 : 9135.96585226059 초 소요
금천구
금천구 : 18340.799824476242 초 소요
영등포구
영등포구 : 5326.548557758331 초 소요
동작구
동작구 : 4717.150148153305 초 소요
관악구
관악구 : 4109.757294416428 초 소요
서초구
서초구 : 6067.040117025375 초 소요
강남구
강남구 : 8441.382312297821 초 소요
송파구
송파구 : 5224.806351900101 초 소요
강동구
강동구 : 4861.623296737671 초 소요
서구
서구 : 3250.1232430934906 초 소요
동구
동구 : 5321.655527830124 초 소요
영도구
영도구 : 4730.277086496353 초 소요
부산진구
부산진구 : 3468.3212172985077 초 소요
동래구


IndexError: list index out of range

동래구 스탑. 서울은 완료. table info Exception. 동래구 '이자카야 히카루'

In [148]:
review_df = review_df.drop(index=review_df[review_df.Region=='동래구'].index)

In [139]:
review_df.to_parquet("./key0-28.parquet")

table info exception 처리

In [168]:
for keyword in key_list[29:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
#             info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
#             region = info_li[0].find_element_by_css_selector("td").text.split('\n')[0]
#             region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
#             category = info_li[2].find_element_by_css_selector("td").text
#             print(category)
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

동래구
동래구 : 7841.559047698975 초 소요
남구
남구 : 8522.637811660767 초 소요
북구
북구 : 3112.7926132678986 초 소요
해운대구
해운대구 : 3808.0706915855408 초 소요
사하구
사하구 : 16646.985271692276 초 소요
금정구
금정구 : 7466.7782373428345 초 소요
연제구
연제구 : 7890.616075992584 초 소요
수영구
수영구 : 5943.614120960236 초 소요
사상구
사상구 : 7597.4245018959045 초 소요
기장군
기장군 : 9941.021861553192 초 소요
수성구
수성구 : 4415.160724878311 초 소요
달서구
달서구 : 13224.85515832901 초 소요
달성군
달성군 : 6410.756312608719 초 소요
미추홀구
미추홀구 : 3339.79656291008 초 소요
연수구
연수구 : 4960.008756875992 초 소요
남동구
남동구 : 5619.977638244629 초 소요
부평구
부평구 : 7013.988264083862 초 소요
계양구
계양구 : 14737.146797180176 초 소요
강화군
강화군 : 6875.127747058868 초 소요
옹진군
옹진군 : 2209.086357355118 초 소요
광산구
광산구 : 11722.385952472687 초 소요
유성구
유성구 : 6308.02047252655 초 소요
대덕구
대덕구 : 14784.212560415268 초 소요
울주군
울주군 : 13008.364349365234 초 소요
반곡동
반곡동 : 3779.5043201446533 초 소요
소담동
소담동 : 119.08561158180237 초 소요
보람동
보람동 : 2059.263782262802 초 소요
대평동
대평동 : 173.95728015899658 초 소요
가람동
가람동 : 1122.4725320339203 초 소요
한솔동
한솔동 : 96.23708391189575 초 소요

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=93.0.4577.82)


과천시 스탑.

In [171]:
review_df = review_df.drop(index=review_df[review_df.Region=='과천시'].index)

In [178]:
review_df.to_parquet("./key0-87.parquet")

In [180]:
review_df.Rating.value_counts()

3    149846
2     43863
1     14640
Name: Rating, dtype: int64

In [183]:
for keyword in key_list[88:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
#             info_li = driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr")
#             region = info_li[0].find_element_by_css_selector("td").text.split('\n')[0]
#             region = region.split()[1] # 도로명 주소의 split 1번째 ex.'서울시 종로구'의 종로구
#             category = info_li[2].find_element_by_css_selector("td").text
#             print(category)
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

과천시
과천시 : 11503.254455327988 초 소요
구리시
구리시 : 9406.849986553192 초 소요
남양주시
남양주시 : 6618.150149822235 초 소요
오산시
오산시 : 4178.778044939041 초 소요
시흥시


InvalidSessionIdException: Message: invalid session id


시흥시 스탑

In [186]:
review_df = review_df.drop(index=review_df[review_df.Region=='시흥시'].index)

In [191]:
review_df.to_parquet("./key0-91.parquet")

In [194]:
for keyword in key_list[92:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

시흥시
시흥시 : 12180.49108338356 초 소요
군포시
군포시 : 10014.443575620651 초 소요
의왕시
의왕시 : 14079.375620126724 초 소요
하남시
하남시 : 7005.840122461319 초 소요
용인시
용인시 : 2855.079498052597 초 소요
파주시
파주시 : 3592.9747495651245 초 소요
이천시
이천시 : 9666.744797468185 초 소요
안성시
안성시 : 5000.992386579514 초 소요
김포시
김포시 : 2691.792966365814 초 소요
화성시
화성시 : 4306.738143920898 초 소요
광주시
광주시 : 7543.617225408554 초 소요
양주시
양주시 : 7218.638829946518 초 소요
포천시
포천시 : 14647.824057340622 초 소요
여주시
여주시 : 7722.16445350647 초 소요
연천군
연천군 : 23923.522154569626 초 소요
가평군
가평군 : 2675.0375049114227 초 소요
양평군
양평군 : 7585.1323454380035 초 소요
춘천시
춘천시 : 5627.09219956398 초 소요
원주시
원주시 : 3571.228436946869 초 소요
강릉시
강릉시 : 3237.783301591873 초 소요
동해시
동해시 : 6802.233869075775 초 소요
태백시
태백시 : 5566.776147842407 초 소요
속초시
속초시 : 6069.8817727565765 초 소요
삼척시
삼척시 : 5027.845634460449 초 소요
홍천군
홍천군 : 7624.310831546783 초 소요
횡성군
횡성군 : 4327.5467076301575 초 소요
영월군
영월군 : 2339.6958968639374 초 소요
평창군
평창군 : 8436.809512138367 초 소요
정선군
정선군 : 4990.615099191666 초 소요
철원군
철원군 : 1671.7281777858734 초 소요
화

KeyboardInterrupt: 

아산시 강제종료.ㅠㅠ

In [195]:
review_df = review_df.drop(index=review_df[review_df.Region=='아산시'].index)

In [203]:
key_list[141]

'아산시'

In [204]:
review_df.to_parquet("./key0-140.parquet")

In [197]:
review_df.Rating.value_counts()

3    172973
2     51523
1     18150
Name: Rating, dtype: int64

In [4]:
review_df

,Region,Rating,Category,Review
0,종로구,3,한정식 / 백반 / 정통 한식,"- 한줄평: 왜 이렇게 평점이 높은지 이해가 가는 집\n- 주문 메뉴: 병어찜 중,..."
1,종로구,3,한정식 / 백반 / 정통 한식,“얼마나 맛있는지 알지 못 할거야”
2,종로구,1,한정식 / 백반 / 정통 한식,"먹으면서도 혼란스러웠고, 리뷰를 쓰는 지금도 혼란스럽다. 어떤 음식점이든 호불호가 ..."
3,종로구,1,한정식 / 백반 / 정통 한식,비린거 싫어하는 사람은 먹지마세요
4,종로구,3,한정식 / 백반 / 정통 한식,#유명세가아깝지않은맛집\n\n유명하지만 다행히도 평일 점심 기준 자리 여유있는편\n...
...,...,...,...,...
242641,보령시,3,탕 / 찌개 / 전골,부모님모시고 여행이라 한끼가 어떤게 좋을지 고민하다. 소꼬리찜 좋아하셔서 찾아갔어요...
242642,보령시,3,정통 중식 / 일반 중식,"대표메뉴 삼춘짬뽕 (12,000) 시켰어요 해물양도 많고 맛도 있음 ! 밀가루 안드..."
242643,보령시,2,해산물 요리,이쪽 지역은 망플 리뷰 불모지 느낌이라 ㅠㅠ 돌아다니며 갈 곳 찾다가 다 비슷해보여...
242644,보령시,2,국수 / 면 요리,직접 도토리면 뽑아서 만드는 집\n6000원 하지만 줄서서 기다려야 됨


In [7]:
for keyword in key_list[141:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

아산시
아산시 : 21580.295776367188 초 소요
서산시
서산시 : 12470.885234355927 초 소요
논산시
논산시 : 5045.150802373886 초 소요
계룡시
계룡시 : 1511.4519581794739 초 소요
당진시
당진시 : 10263.540743350983 초 소요
금산군
금산군 : 6080.861167907715 초 소요
부여군
부여군 : 5534.593491315842 초 소요
서천군
서천군 : 2896.2599070072174 초 소요
청양군
청양군 : 3873.650764465332 초 소요
홍성군
홍성군 : 3734.624661207199 초 소요
예산군
예산군 : 8029.048336267471 초 소요
태안군
태안군 : 7754.300993204117 초 소요
전주시
전주시 : 2382.8585138320923 초 소요
군산시
군산시 : 3055.361421585083 초 소요
익산시
익산시 : 5469.89413189888 초 소요
정읍시
정읍시 : 1433.0769822597504 초 소요
남원시
남원시 : 2664.139569044113 초 소요
김제시
김제시 : 1265.0330810546875 초 소요
완주군
완주군 : 1531.2024538516998 초 소요
진안군
진안군 : 442.9729986190796 초 소요
무주군
무주군 : 1124.719453573227 초 소요
장수군
장수군 : 442.26396441459656 초 소요
임실군


InvalidSessionIdException: Message: invalid session id


In [14]:
review_df = review_df.drop(index=review_df[review_df.Region=='임실군'].index)

In [15]:
review_df

,Region,Rating,Category,Review
0,종로구,3.0,한정식 / 백반 / 정통 한식,"- 한줄평: 왜 이렇게 평점이 높은지 이해가 가는 집\n- 주문 메뉴: 병어찜 중,..."
1,종로구,3.0,한정식 / 백반 / 정통 한식,“얼마나 맛있는지 알지 못 할거야”
2,종로구,1.0,한정식 / 백반 / 정통 한식,"먹으면서도 혼란스러웠고, 리뷰를 쓰는 지금도 혼란스럽다. 어떤 음식점이든 호불호가 ..."
3,종로구,1.0,한정식 / 백반 / 정통 한식,비린거 싫어하는 사람은 먹지마세요
4,종로구,3.0,한정식 / 백반 / 정통 한식,#유명세가아깝지않은맛집\n\n유명하지만 다행히도 평일 점심 기준 자리 여유있는편\n...
...,...,...,...,...
247977,무주군,3.0,카페 / 디저트,무주 핫플이라 해서 와 봤는데 넓은 실내에 예쁜 인테리어\n\n한번 더 방문 하고싶...
247978,장수군,3.0,닭 / 오리 요리,계곡에서 신나게 놀고 배가 고파서 밥집을 찾다가\n몸보신도 할겸 닭백숙을 먹으러 초...
247979,장수군,2.0,탕 / 찌개 / 전골,장수에서 저녁 8시에 식사를 하는게 이렇게 어려운 일인줄 몰랐다. 문 연 식당을 찾...
247980,장수군,3.0,해산물 요리,"이름은 분식집이지만 메뉴는 올갱이 수제비, 올갱이 칼국수, 올갱이 탕 이렇게 세가지..."


In [16]:
key_list[163]

'임실군'

In [17]:
review_df.to_parquet("./key0-162.parquet")

In [18]:
review_df.Rating.value_counts()

3.0    176452
2.0     52758
1.0     18772
Name: Rating, dtype: int64

In [21]:
for keyword in key_list[163:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

임실군
임실군 : 790.9175362586975 초 소요
순창군
순창군 : 1434.5734777450562 초 소요
고창군
고창군 : 7506.7603895664215 초 소요
부안군
부안군 : 3653.977197408676 초 소요
목포시
목포시 : 13459.12794828415 초 소요
여수시
여수시 : 9312.741170167923 초 소요
순천시
순천시 : 8173.956686258316 초 소요
나주시
나주시 : 3904.6968581676483 초 소요
광양시
광양시 : 6493.412637233734 초 소요
담양군
담양군 : 1929.0446972846985 초 소요
곡성군
곡성군 : 538.1150500774384 초 소요
구례군
구례군 : 562.4800338745117 초 소요
고흥군


InvalidSessionIdException: Message: invalid session id


In [23]:
review_df = review_df.drop(index=review_df[review_df.Region=='고흥군'].index)

In [24]:
review_df

,Region,Rating,Category,Review
0,종로구,3.0,한정식 / 백반 / 정통 한식,"- 한줄평: 왜 이렇게 평점이 높은지 이해가 가는 집\n- 주문 메뉴: 병어찜 중,..."
1,종로구,3.0,한정식 / 백반 / 정통 한식,“얼마나 맛있는지 알지 못 할거야”
2,종로구,1.0,한정식 / 백반 / 정통 한식,"먹으면서도 혼란스러웠고, 리뷰를 쓰는 지금도 혼란스럽다. 어떤 음식점이든 호불호가 ..."
3,종로구,1.0,한정식 / 백반 / 정통 한식,비린거 싫어하는 사람은 먹지마세요
4,종로구,3.0,한정식 / 백반 / 정통 한식,#유명세가아깝지않은맛집\n\n유명하지만 다행히도 평일 점심 기준 자리 여유있는편\n...
...,...,...,...,...
251127,구례군,3.0,카페 / 디저트,원두맛 좋아요~ 사장님도 친절하시고 안알려졌으면 좋겠지만 또 잘 됐으면 좋겠어요
251128,구례군,3.0,카페 / 디저트,지리산과하나되기 카페\n커피맛 좋네요
251129,구례군,3.0,한정식 / 백반 / 정통 한식,오삼볶음이 맛있다고 해서 갔지만 1인 손님은 낙지볶음만 가능하다고 하셔서 낙지볶음 ...
251130,구례군,3.0,고기 요리,여길 먹어보지 않고 닭도리탕을 논하지말라\n살면서 수도없을 만큼 계곡옆 닭도리탕 집...


In [28]:
key_list[175]

'고흥군'

In [29]:
review_df.to_parquet("./key0-174.parquet")

In [32]:
review_df.Rating.value_counts()

3.0    178643
2.0     53370
1.0     19119
Name: Rating, dtype: int64

In [35]:
for keyword in key_list[175:]:
    search_time = time.time()
    print("===============")
    print(keyword)
    driver.get("https://www.mangoplate.com/search/"+keyword)
    time.sleep(5)
    for page in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div > div > section > div.paging-container > p.paging > a"):
        page.click()
        time.sleep(2)
        for each_restaurant in driver.find_elements_by_css_selector("a > span.title.ng-binding"):
            ActionChains(driver).key_down(Keys.CONTROL).click(each_restaurant).perform()
            driver.switch_to.window(driver.window_handles[-1])

            for info_table in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr"):
                if info_table.find_element_by_css_selector("th").text == '주소':
                    region = info_table.find_element_by_css_selector("td").text.split("\n")[0]
                    region = region.split()[1]
                elif info_table.find_element_by_css_selector("th").text == '음식 종류':
                    category = info_table.find_element_by_css_selector("td").text

            start_time = time.time()
            while (time.time() - start_time) < 601:  # 일정 사이트 자체에 더보기 오류(자체적 오류)를 해결 위해 루프 시간 10분 제한
                try: # more button click                  
                    more_button = driver.find_element_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton")
                    ActionChains(driver).move_to_element(more_button).click(more_button).perform()
                    time.sleep(1)
                except ElementNotInteractableException:
                    break
                except NoSuchElementException:
                    break
            review_list, rating_list = [], []
            for review in driver.find_elements_by_css_selector("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li"):
                each_review = review.find_element_by_css_selector("a > div.RestaurantReviewItem__ReviewContent > div > p").text
                each_rating = review.find_element_by_css_selector("a > div.RestaurantReviewItem__Rating > span").text
                review_list.append(each_review)
                rating_list.append(rating_dict[each_rating])
            review_df = review_df.append(pd.DataFrame(data={'Region': [region]*len(rating_list), 'Rating':rating_list, 'Category':[category]*len(rating_list), 'Review':review_list}), ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print(f'{keyword} : {time.time() - search_time} 초 소요')

고흥군
고흥군 : 2759.9574756622314 초 소요
보성군
보성군 : 2971.9820272922516 초 소요
화순군
화순군 : 7806.39781665802 초 소요
장흥군
장흥군 : 2178.378051519394 초 소요
강진군
강진군 : 3362.921466588974 초 소요
해남군
해남군 : 4123.285399675369 초 소요
영암군
영암군 : 2131.150342941284 초 소요
무안군
무안군 : 1110.8006036281586 초 소요
함평군
함평군 : 760.5940067768097 초 소요
영광군
영광군 : 2908.0075209140778 초 소요
장성군
장성군 : 855.6790864467621 초 소요
완도군
완도군 : 1492.307898759842 초 소요
진도군
진도군 : 211.9598045349121 초 소요
신안군
신안군 : 184.9959692955017 초 소요
포항시
포항시 : 12218.035337209702 초 소요
경주시
경주시 : 5650.409192562103 초 소요
김천시
김천시 : 2461.666468858719 초 소요
안동시
안동시 : 10821.869729757309 초 소요
구미시
구미시 : 17745.922502994537 초 소요
영주시
영주시 : 1962.3891077041626 초 소요
영천시
영천시 : 2323.141047000885 초 소요
상주시
상주시 : 3004.7178909778595 초 소요
문경시
문경시 : 7532.047028064728 초 소요
경산시
경산시 : 17422.901475429535 초 소요
군위군
군위군 : 745.8302614688873 초 소요
의성군
의성군 : 713.4115753173828 초 소요
청송군
청송군 : 2099.286959171295 초 소요
영양군
영양군 : 682.1355392932892 초 소요
영덕군
영덕군 : 1115.4678485393524 초 소요
청도군
청도군 : 4240.347128629684 초 소요


In [47]:
review_df.Region.value_counts()[-75:]

달빛1로       33
화천군        31
장성군        30
창녕군        29
산청군        29
           ..
대평1길        1
아름서길        1
국책연구원1로     1
반곡로         1
보람동로        1
Name: Region, Length: 75, dtype: int64

In [51]:
review_df.Region.value_counts()[-70:]

괴산군        29
새롬중앙로      28
고흥군        27
영암군        26
예천군        25
           ..
대평1길        1
아름서길        1
국책연구원1로     1
반곡로         1
보람동로        1
Name: Region, Length: 70, dtype: int64

In [52]:
review_df.Region.value_counts()[-65:]

노을3로       25
합천군        25
강진군        25
절재로        24
장흥군        24
           ..
대평1길        1
아름서길        1
국책연구원1로     1
반곡로         1
보람동로        1
Name: Region, Length: 65, dtype: int64

In [54]:
review_df.Region.value_counts()[-60:]

진안군        20
영광군        20
부강면        20
순창군        20
영동군        20
곡성군        19
함안군        19
거창군        19
함양군        19
임실군        16
함평군        15
도움3로       15
봉화군        14
도움1로       14
성주군        13
만남로        12
영양군        11
마음로        11
호려울로       10
바른7길       10
신안군         9
보듬7로        9
남세종로        9
갈매로         9
의령군         9
청양군         9
새롬로         8
아름서1길       8
다정중앙로       7
가름로         7
소정면         7
고운서길        7
연기면         7
전동면         6
군위군         5
의성군         5
법원2로        4
장수군         4
나성북1로       4
나성북로        3
보듬4로        3
고령군         2
새롬북로        2
바른3길        2
대평3길        2
누리로         2
다솜로         2
고운서1길       2
금송로         2
국책연구원3로     2
고운한옥1길      2
세종로         2
누리1로        1
전의면         1
다정5길        1
대평1길        1
아름서길        1
국책연구원1로     1
반곡로         1
보람동로        1
Name: Region, dtype: int64

In [184]:
driver.quit()

In [49]:
review_df.Region.value_counts()[:-75]

강남구     27582
마포구     13611
용산구     12652
종로구     12550
서대문구    10047
        ...  
양구군        33
증평군        33
남구         33
금산군        33
진도군        33
Name: Region, Length: 186, dtype: int64

# 지역명을 확인해 보니, 세종특별자치시 에 통일성이 사라져, 문제 발생. (다른 광역시의 경우 구 로 나눌 수 있는데, 세종시는 구가 없어서, 도로명 주소인 OO로 로 파싱됨.)

### OO로, OO길 -> 처리

In [82]:
review_df.Region[review_df.Region.str.contains('길')].value_counts()

한천로139가길    46
바른7길        10
아름서1길        8
고운서길         7
고운서1길        2
고운한옥1길       2
바른3길         2
대평3길         2
아름서길         1
대평1길         1
다정5길         1
Name: Region, dtype: int64

In [84]:
review_df.Region[review_df.Region=='한천로139가길'] = '강북구'

C:\Users\dongi\AppData\Local\Temp/ipykernel_25100/951000272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.Region[review_df.Region=='한천로139가길'] = '강북구'


In [86]:
(review_df.Region=='한천로139가길').sum()

0

In [87]:
sejong_list = ['바른7길', '아름서1길', '고운서길', '고운서1길', '고운한옥1길', '바른3길', '대평3길', '아름서길', '대평1길', '다정5길']

In [92]:
review_df.Region[review_df.Region.isin(sejong_list)] = '세종특별자치시'

C:\Users\dongi\AppData\Local\Temp/ipykernel_25100/3822195945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.Region[review_df.Region.isin(sejong_list)] = '세종특별자치시'


In [96]:
sejong_list = review_df.Region[review_df.Region.str.contains('로')].value_counts().keys()

In [99]:
sejong_list

Index(['종로구', '구로구', '한누리대로', '시청대로', '달빛로', '보듬3로', '도움8로', '나성로', '달빛1로',
       '새롬중앙로', '노을3로', '절재로', '도움3로', '도움1로', '만남로', '마음로', '호려울로', '남세종로',
       '갈매로', '보듬7로', '새롬로', '다정중앙로', '가름로', '나성북1로', '법원2로', '나성북로', '보듬4로',
       '누리로', '국책연구원3로', '새롬북로', '다솜로', '세종로', '금송로', '국책연구원1로', '누리1로',
       '보람동로', '반곡로'],
      dtype='object')

In [104]:
review_df.Region[review_df.Region.isin(sejong_list[2:])] = '세종특별자치시'

C:\Users\dongi\AppData\Local\Temp/ipykernel_25100/2040306785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.Region[review_df.Region.isin(sejong_list[2:])] = '세종특별자치시'


In [105]:
review_df.Region.value_counts()

강남구     27582
마포구     13611
용산구     12652
종로구     12550
서대문구    10047
        ...  
군위군         5
의성군         5
장수군         4
고령군         2
전의면         1
Name: Region, Length: 216, dtype: int64

In [111]:
sejong_list = review_df.Region[review_df.Region.str.contains('면')].value_counts().keys()

In [113]:
sejong_list

Index(['금남면', '장군면', '연서면', '부강면', '연기면', '소정면', '전동면'], dtype='object')

In [114]:
review_df.Region[review_df.Region.isin(sejong_list)] = '세종특별자치시'

C:\Users\dongi\AppData\Local\Temp/ipykernel_25100/3822195945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.Region[review_df.Region.isin(sejong_list)] = '세종특별자치시'


In [116]:
review_df.Region.value_counts()[-50:]

옥천군    51
상주시    50
무안군    50
무주군    49
울릉군    47
칠곡군    45
완도군    42
영천시    42
청도군    41
서천군    40
김제시    39
영덕군    38
보은군    35
청송군    34
증평군    33
진도군    33
양구군    33
남구     33
금산군    33
화천군    31
장성군    30
산청군    29
창녕군    29
괴산군    29
고흥군    27
영암군    26
강진군    25
예천군    25
합천군    25
장흥군    24
순창군    20
진안군    20
영광군    20
영동군    20
함안군    19
함양군    19
거창군    19
곡성군    19
임실군    16
함평군    15
봉화군    14
성주군    13
영양군    11
의령군     9
신안군     9
청양군     9
군위군     5
의성군     5
장수군     4
고령군     2
Name: Region, dtype: int64

In [133]:
review_df.Region[review_df.Region.str.contains('읍')].value_counts()

조치원읍    591
정읍시      74
Name: Region, dtype: int64

In [134]:
review_df.Region[review_df.Region=='조치원읍'] = '세종특별자치시'

C:\Users\dongi\AppData\Local\Temp/ipykernel_25100/377807891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.Region[review_df.Region=='조치원읍'] = '세종특별자치시'


In [162]:
review_df.Region.value_counts()

강남구     27582
마포구     13611
용산구     12652
종로구     12550
서대문구    10047
        ...  
청양군         9
군위군         5
의성군         5
장수군         4
고령군         2
Name: Region, Length: 207, dtype: int64

In [166]:
review_df.Region.value_counts().keys().isin(key_list)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [171]:
key_list[pd.Series(key_list).isin(review_df.Region.value_counts().keys())==False]

array(['반곡동', '소담동', '보람동', '대평동', '가람동', '한솔동', '나성동', '새롬동', '다정동',
       '어진동', '종촌동', '고운동', '아름동', '도담동', '조치원읍', '연기면', '연동면', '부강면',
       '금남면', '장군면', '연서면', '전의면', '전동면', '소정면'], dtype=object)

아하, key_list에서도 잘못 슬라이싱 되었구나...

key list: 231개

현재 review_df의 지역 (207개) + 잘못 파싱한 세종시 지역 (24) = 231

In [50]:
len(key_list)

231

# 종료 (ToDo : 중복 제거)

In [185]:
review_df

,Region,Rating,Category,Review
0,종로구,3.0,한정식 / 백반 / 정통 한식,"- 한줄평: 왜 이렇게 평점이 높은지 이해가 가는 집\n- 주문 메뉴: 병어찜 중,..."
1,종로구,3.0,한정식 / 백반 / 정통 한식,“얼마나 맛있는지 알지 못 할거야”
2,종로구,1.0,한정식 / 백반 / 정통 한식,"먹으면서도 혼란스러웠고, 리뷰를 쓰는 지금도 혼란스럽다. 어떤 음식점이든 호불호가 ..."
3,종로구,1.0,한정식 / 백반 / 정통 한식,비린거 싫어하는 사람은 먹지마세요
4,종로구,3.0,한정식 / 백반 / 정통 한식,#유명세가아깝지않은맛집\n\n유명하지만 다행히도 평일 점심 기준 자리 여유있는편\n...
...,...,...,...,...
267543,서귀포시,3.0,고기 요리,제주도 올레길 7코스종점에 진짜 흑돼지맛집이 있네요.\n고기.반찬 모두 맛있구요.\...
267544,서귀포시,3.0,고기 요리,고기는 최고.반찬은 집에서먹는 맛이고요 ..써비스 친절도 최고입니다
267545,서귀포시,3.0,카페 / 디저트,애플파이와 하겐다즈 바닐라아이스크림\n8500 원 넘나 비싸지만 최고오브 최고\n사...
267546,서귀포시,2.0,카페 / 디저트,"60, 70년대 향취 가득한 음악들과 카페 분위기가 잘 어울리는 카페. 디저트나 카..."


In [186]:
review_df.Rating.value_counts()

3.0    189847
2.0     56966
1.0     20735
Name: Rating, dtype: int64

In [ ]:
review_df.to_parquet("./key_all.parquet")